In [183]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output
import json
import os
import pandas as pd


In [191]:
def pading(n):
    if n < 10:
        return '000' + str(n)
    elif n < 100:
        return '00' + str(n)
    else:
        return '0' + str(n)
        

In [185]:
def get_game(season, game_type, game_id, play_id):
    
    year=season.split('-')[0]
    file= f'{year}{game_type}{pading(game_id)}.json'

    #entries = os.listdir("raw")
    #entries = [e for e in entries if e.startswith(id_start)]

    if not os.path.exists('raw' + '/' +file): 
        clear_output()
        return
        
    with open('raw' + '/' +file , 'r') as game_file:               
            game_json = json.load(game_file)
    
    gameData = game_json['gameData']
    #print(gameData)
    print(gameData["datetime"]["dateTime"])
    print('Game ID:', game_id, ';', gameData["teams"]["home"]["triCode"],'(home) vs',gameData["teams"]["away"]["triCode"],'(away)')

    data = {
    "Home":[],
    "Away":[],
    }

    liveData = game_json['liveData']
    data["Home"].append(gameData["teams"]["home"]["triCode"])
    data["Away"].append(gameData["teams"]["away"]["triCode"])

    data["Home"].append(liveData["boxscore"]["teams"]["home"]["teamStats"]["teamSkaterStats"]["goals"])
    data["Away"].append(liveData["boxscore"]["teams"]["away"]["teamStats"]["teamSkaterStats"]["goals"])
   
    data["Home"].append(liveData["boxscore"]["teams"]["home"]["teamStats"]["teamSkaterStats"]["shots"])
    data["Away"].append(liveData["boxscore"]["teams"]["away"]["teamStats"]["teamSkaterStats"]["shots"])
  

    data["Home"].append(liveData["linescore"]["shootoutInfo"]["home"]["scores"])
    data["Away"].append(liveData["linescore"]["shootoutInfo"]["away"]["scores"])


    data["Home"].append(liveData["linescore"]["shootoutInfo"]["home"]["attempts"])
    data["Away"].append(liveData["linescore"]["shootoutInfo"]["away"]["attempts"])

    df = pd.DataFrame(data, index=['Teams', 'Goals', 'SoG', 'SO Goals', 'SO Attempts' ])

    print('\n')
    print(df)
    print('\n\n')

    # display the image and add data on it with the coordinates
    
    all_plays = liveData['plays']['allPlays']
    for p in all_plays:
        eventIdx = p['about']['eventIdx']
        if eventIdx == play_id:
            print(json.dumps(p,indent=2))
            break
    #return game_json
    

In [190]:
interact(
    get_game,
    
    season = widgets.Dropdown(
        options=['2016-2017','2017-2018', '2018-2019', '2019-2020', '2020-2021'],
        value='2016-2017',
        description='Select a season'
    ),
    game_type=widgets.ToggleButtons(
        options = ['02', '03'],
        value='02',
        description='Select game type'
    ),
    game_id = widgets.IntSlider(min=1, max=999, step=1, value=1),
    play_id = widgets.IntSlider(min=1, max=300, step=1, value=1)
)

interactive(children=(Dropdown(description='Select a season', options=('2016-2017', '2017-2018', '2018-2019', …

<function __main__.get_game(season, game_type, game_id, play_id)>